# Live Trading Dashboard

This notebook creates a real-time dashboard for monitoring your Bitcoin trading bot signals and performance.

In [ ]:
# Setup and imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import json
from IPython.display import clear_output, display, HTML
import warnings
warnings.filterwarnings('ignore')

# Configuration
API_BASE_URL = "http://localhost:8000"
UPDATE_INTERVAL = 30  # seconds

# Initialize data storage
signal_history = []
price_history = []

print("🚀 Bitcoin Trading Bot Live Dashboard")
print("📊 Make sure your API is running at http://localhost:8000")
print("🔄 Dashboard will update every 30 seconds")

In [ ]:
# Dashboard functions
def get_current_signal():
    """Fetch current trading signal from API"""
    try:
        response = requests.get(f"{API_BASE_URL}/signal", timeout=10)
        if response.status_code == 200:
            return response.json()
        else:
            return None
    except Exception as e:
        print(f"Error fetching signal: {e}")
        return None

def get_api_health():
    """Check API health"""
    try:
        response = requests.get(f"{API_BASE_URL}/health", timeout=5)
        if response.status_code == 200:
            return response.json()
        else:
            return None
    except Exception as e:
        return None

def update_dashboard():
    """Update the dashboard with latest data"""
    clear_output(wait=True)
    
    # Get current data
    health = get_api_health()
    signal_data = get_current_signal()
    
    # Display header
    print("=" * 80)
    print("🚀 BITCOIN TRADING BOT - LIVE DASHBOARD")
    print("=" * 80)
    print(f"📅 Last Update: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # API Status
    if health:
        status_emoji = "✅" if health['status'] == 'healthy' else "❌"
        print(f"🔌 API Status: {status_emoji} {health['status'].upper()}")
        print(f"🤖 Model Loaded: {'✅' if health['model_loaded'] else '❌'}")
        print(f"📊 Data Points: {health['data_points']}")
    else:
        print("🔌 API Status: ❌ OFFLINE")
        return
    
    # Current Signal
    print("\n" + "=" * 40)
    print("🎯 CURRENT TRADING SIGNAL")
    print("=" * 40)
    
    if signal_data:
        signal = signal_data['signal']
        confidence = signal_data['confidence']
        price = signal_data['current_price']
        
        # Signal emoji
        signal_emoji = "🟢" if signal == "BUY" else "🔴" if signal == "SELL" else "🟡"
        confidence_bars = "█" * int(confidence * 10)
        
        print(f"📈 Signal: {signal_emoji} {signal}")
        print(f"🎯 Confidence: {confidence:.2%} [{confidence_bars}]")
        print(f"💰 Current Price: ${price:,.2f}")
        print(f"📊 Predicted Return: {signal_data['predicted_return']:.2%}")
        
        # Technical Indicators
        print(f"\n📋 Technical Indicators:")
        indicators = signal_data['technical_indicators']
        if indicators['rsi']:
            rsi_status = "🔴 Overbought" if indicators['rsi'] > 70 else "🟢 Oversold" if indicators['rsi'] < 30 else "🟡 Neutral"
            print(f"   RSI: {indicators['rsi']:.1f} {rsi_status}")
        
        if indicators['sma_20'] and indicators['sma_50']:
            ma_trend = "🟢 Bullish" if indicators['sma_20'] > indicators['sma_50'] else "🔴 Bearish"
            print(f"   MA Trend: {ma_trend}")
        
        # Store history
        signal_history.append({
            'timestamp': datetime.now(),
            'signal': signal,
            'confidence': confidence,
            'price': price
        })
        
        # Keep only last 100 records
        if len(signal_history) > 100:
            signal_history.pop(0)
            
    else:
        print("❌ Unable to fetch signal data")
    
    # Signal History Chart
    if len(signal_history) > 1:
        print(f"\n📈 SIGNAL HISTORY (Last {len(signal_history)} signals)")
        print("=" * 50)
        
        df_history = pd.DataFrame(signal_history)
        
        # Count signals
        signal_counts = df_history['signal'].value_counts()
        print("📊 Signal Distribution:")
        for signal, count in signal_counts.items():
            percentage = count / len(df_history) * 100
            print(f"   {signal}: {count} ({percentage:.1f}%)")
        
        # Price trend
        price_change = (df_history['price'].iloc[-1] - df_history['price'].iloc[0]) / df_history['price'].iloc[0] * 100
        trend_emoji = "📈" if price_change > 0 else "📉"
        print(f"\n💹 Price Trend: {trend_emoji} {price_change:+.2f}%")
        
        # Recent signals
        print(f"\n🕐 Recent Signals:")
        recent = df_history.tail(5)
        for _, row in recent.iterrows():
            time_str = row['timestamp'].strftime('%H:%M:%S')
            signal_emoji = "🟢" if row['signal'] == "BUY" else "🔴" if row['signal'] == "SELL" else "🟡"
            print(f"   {time_str}: {signal_emoji} {row['signal']} (${row['price']:,.0f})")

# Start dashboard
print("Starting dashboard in 3 seconds...")
time.sleep(3)

In [ ]:
# Run live dashboard (run this cell to start)
try:
    while True:
        update_dashboard()
        time.sleep(UPDATE_INTERVAL)
except KeyboardInterrupt:
    print("\n\n🛑 Dashboard stopped by user")
    
    # Save history
    if signal_history:
        df_final = pd.DataFrame(signal_history)
        filename = f"signal_history_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df_final.to_csv(filename, index=False)
        print(f"💾 Signal history saved to: {filename}")